In [1]:
!pip install -q unsloth==2025.6.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 7.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.2/160.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 55.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 6.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 24.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 21.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 1.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57

In [2]:
%cd /content

/content


In [3]:
import unsloth
import datasets
import numpy as np
import unsloth
import torch
import trl 
import transformers


print("datasets.__version__", datasets.__version__)
print("numpy.__version__", np.__version__)
print("unsloth.__version__", unsloth.__version__)
print("torch.__version__", torch.__version__)
print("transformers.__version__", transformers.__version__)
print("trl.__version__", trl.__version__)

# datasets.__version__ 3.6.0
# numpy.__version__ 1.26.4
# unsloth.__version__ 2025.6.3
# torch.__version__ 2.7.0+cu126
# transformers.__version__ 4.51.3
# trl.__version__ 0.19.0

NotImplementedError: Unsloth currently only works on NVIDIA GPUs and Intel GPUs.

In [ ]:
# To temporary Model hub
from huggingface_hub import HfApi
from huggingface_hub import login
# Initialize API
login("<your_huggingface_token>")
api = HfApi()

In [ ]:
from huggingface_hub import HfApi
from huggingface_hub import snapshot_download
import os
api = HfApi()

repo_id = "danhtran2mind/Qwen-3-0.6B-Instruct-Vi-Medical-LoRA"
save_path = "Qwen-3-0.6B-Instruct-Vi-Medical-LoRA"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Download the dataset
snapshot_download(repo_id=repo_id, repo_type="model", local_dir=save_path)

In [ ]:
import numpy as np
from datasets import concatenate_datasets, load_dataset

from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only  
import torch

from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq

In [ ]:
dataset_id = "tmnam20/ViMedAQA"
subset = "disease"
dataset = load_dataset(dataset_id, subset)

In [ ]:
# Merge training and validation samples
test_samples = concatenate_datasets([dataset['validation'],
                                         dataset['test']])

# Shuffle the merged dataset with a fixed random seed
training_samples = dataset['train']
test_samples = test_samples.shuffle(seed=42)

print("Number of training samples:", training_samples.num_rows)
print("Number of test samples:", test_samples.num_rows)

In [9]:
dataset['train'][0]

{'question_idx': 'disease_13345',
 'question': 'Khi soi bằng đèn thường, triệu chứng điển hình của viêm mống mắt là gì?',
 'answer': 'Đồng tử 2 bên không đều nhau, đồng tử bên bệnh nhỏ hơn bên bình thường.',
 'context': '5.1 Nhìn về mặt đại thể (khi soi bằng đèn thường) - Đồng tử 2 bên không đều nhau (bên bệnh nhỏ hơn) Có thể phát hiện đồng tử co bên bệnh lý co nhiều hơn bên bình thường, điều này có thể do viêm nhiều dẫn đến dính thành sau mống mắt lại với nhau và với thuỷ tinh thể. Thông thường kích thường đồng tử là 2 – 3 mm, đều 2 bên. - Mủ ở đáy mống mắt (mủ tiền phòng) Đây là hiện tượng các tế bào viêm cùng với chất tiết (chứa nhiều fibrin). Mủ nằm ở dưới là do nguyên nhân trọng lực. 5.2 Khám mắt qua đèn khe: (split lamp) Đây là công cụ chuyên dụng của bác sĩ chuyên khoa Mắt. Qua đèn khe, bác sĩ có thể phát hiện: - Các tế bào nằm giữa giác mạc và mống mắt Số lượng tế bào này trên diện tích 1 mm² giúp phân độ nặng của viêm màng bồ đào. - Độ mờ thuỷ dịch Như đã miêu tả, thuỷ dịch là

In [ ]:
from unsloth import FastLanguageModel
import torch

model_id = "Qwen/Qwen3-0.6B"
# Load the model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)


In [ ]:
def convert_squad_sample_to_conversation(sample):
    # get the question and context for this sample
    question = sample['question']
    context = sample['context']

    answers = sample['answer']
    if len(answers) == 0 :
      answer = "The context does not provide an answer..."
    else:
      answer = sample['answer']

    # now we define an initial model prompt defining the task and giving the model the context passage
    instruction_prompt_template = '''Bạn là một trợ lý hữu ích được giao nhiệm vụ trích xuất các đoạn văn trả lời câu hỏi của người dùng từ một ngữ cảnh cho trước. Xuất ra các đoạn văn chính xác từng từ một trả lời câu hỏi của người dùng. Không xuất ra bất kỳ văn bản nào khác ngoài các đoạn văn trong ngữ cảnh. Xuất ra lượng tối thiểu để trả lời câu hỏi, ví dụ chỉ 2-3 từ từ đoạn văn. Nếu không thể tìm thấy câu trả lời trong ngữ cảnh, xuất ra 'Ngữ cảnh không cung cấp câu trả lời...'

    Ngữ cảnh: {context}'''

    # now we'll convert these into a list of messages for our conversation
    messages = [
        {"role": "system", "content": instruction_prompt_template.format(context=context)},
        {"role": "user", "content": question},
        {"role": "assistant", "content": answer}
    ]
    
    sample_conversation = tokenizer.apply_chat_template(messages, tokenize=False)
    return {"text": sample_conversation, "messages": messages, "answer": answer}


In [ ]:
conversation_training_samples = training_samples.map(convert_squad_sample_to_conversation)
conversation_test_samples = test_samples.map(convert_squad_sample_to_conversation)

In [13]:
conversation_test_samples[0]['answer']

'Nếu bạn có các triệu chứng nghi ngờ của nhiễm nấm da chân, hãy liên hệ với bác sĩ ngay để được tham vấn điều trị sớm. Điều này sẽ giúp bạn kiểm soát bệnh tình và ngăn ngừa các biến chứng nghiêm trọng.'

In [14]:
conversation_test_samples[0]['messages']

[{'content': "Bạn là một trợ lý hữu ích được giao nhiệm vụ trích xuất các đoạn văn trả lời câu hỏi của người dùng từ một ngữ cảnh cho trước. Xuất ra các đoạn văn chính xác từng từ một trả lời câu hỏi của người dùng. Không xuất ra bất kỳ văn bản nào khác ngoài các đoạn văn trong ngữ cảnh. Xuất ra lượng tối thiểu để trả lời câu hỏi, ví dụ chỉ 2-3 từ từ đoạn văn. Nếu không thể tìm thấy câu trả lời trong ngữ cảnh, xuất ra 'Ngữ cảnh không cung cấp câu trả lời...'\n\n    Ngữ cảnh: Những mẹo này có thể giúp bạn tránh bị nấm da chân hoặc giảm bớt các triệu chứng nếu nhiễm nấm xảy ra: - Giữ chân khô ráo, đặc biệt là giữa các ngón chân. Đi chân trần để chân thoát khí nhiều nhất có thể khi ở nhà. Lau khô các kẽ ngón chân sau khi tắm.\n- Thay tất thường xuyên. Nếu chân bạn ra nhiều mồ hôi, hãy thay tất hai lần một ngày.\n- Đi giày nhẹ, thông thoáng. Tránh giày làm bằng chất liệu tổng hợp, chẳng hạn như nhựa vinyl hoặc cao su.\n- Mang thay đổi các đôi giày. Đừng mang cùng một đôi mỗi ngày để giày c

In [12]:
conversation_test_samples[0]['text']

"<|im_start|>system\nBạn là một trợ lý hữu ích được giao nhiệm vụ trích xuất các đoạn văn trả lời câu hỏi của người dùng từ một ngữ cảnh cho trước. Xuất ra các đoạn văn chính xác từng từ một trả lời câu hỏi của người dùng. Không xuất ra bất kỳ văn bản nào khác ngoài các đoạn văn trong ngữ cảnh. Xuất ra lượng tối thiểu để trả lời câu hỏi, ví dụ chỉ 2-3 từ từ đoạn văn. Nếu không thể tìm thấy câu trả lời trong ngữ cảnh, xuất ra 'Ngữ cảnh không cung cấp câu trả lời...'\n\n    Ngữ cảnh: Những mẹo này có thể giúp bạn tránh bị nấm da chân hoặc giảm bớt các triệu chứng nếu nhiễm nấm xảy ra: - Giữ chân khô ráo, đặc biệt là giữa các ngón chân. Đi chân trần để chân thoát khí nhiều nhất có thể khi ở nhà. Lau khô các kẽ ngón chân sau khi tắm.\n- Thay tất thường xuyên. Nếu chân bạn ra nhiều mồ hôi, hãy thay tất hai lần một ngày.\n- Đi giày nhẹ, thông thoáng. Tránh giày làm bằng chất liệu tổng hợp, chẳng hạn như nhựa vinyl hoặc cao su.\n- Mang thay đổi các đôi giày. Đừng mang cùng một đôi mỗi ngày để

In [ ]:
model = FastLanguageModel.get_peft_model(  
    model,  
    r=16,  
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",  
                    "gate_proj", "up_proj", "down_proj"],  
    lora_alpha=16,  
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  
    random_state=42,  
    use_rslora=False,  
    loftq_config=None
)

In [ ]:
arguments = TrainingArguments(  
                resume_from_checkpoint="./Qwen-3-0.6B-Instruct-Vi-Medical-LoRA",
                per_device_train_batch_size=8,  
                per_device_eval_batch_size=8,  
                gradient_accumulation_steps=4,  
                eval_strategy="steps",
                eval_steps=100,
                logging_steps=50,
                save_steps=100,
                warmup_steps=30,
                save_total_limit=4,
                num_train_epochs=50, # 5
                # max_steps=80,
                save_strategy="steps",
                metric_for_best_model="eval_loss",
                learning_rate=2e-4,  
                fp16=not is_bfloat16_supported(),  
                bf16=is_bfloat16_supported(),  
                optim="adamw_8bit",  
                weight_decay=0.01,  
                lr_scheduler_type="linear",  
                seed=42,  
                output_dir="Qwen-3-0.6B-Instruct-Vi-Medical-LoRA",  
                report_to="none",
                load_best_model_at_end=True,  # Load weights with lowest val loss
        		greater_is_better=False,
            )

In [ ]:
# Define Trainer
trainer = SFTTrainer(  
    model=model,  
    tokenizer=tokenizer,  
    train_dataset=conversation_training_samples,  
    eval_dataset=conversation_test_samples,  
    # dataset_text_field="text",
    max_seq_length=2048,  
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),  
    dataset_num_proc=2,  
    packing=False,  # Can make training 5x faster for short sequences.  
    args=arguments
)

In [16]:
!ls -a Qwen-3-0.6B-Instruct-Vi-Medical-LoRA

.		  checkpoint-15800  checkpoint-19800  checkpoint-7938
..		  checkpoint-15900  checkpoint-19845  checkpoint-8600
.cache		  checkpoint-16000  checkpoint-20500  checkpoint-8700
checkpoint-11800  checkpoint-16100  checkpoint-20600  checkpoint-8800
checkpoint-11900  checkpoint-17000  checkpoint-20700  checkpoint-8820
checkpoint-11907  checkpoint-17100  checkpoint-20727  checkpoint-9600
checkpoint-13000  checkpoint-17200  checkpoint-6400   checkpoint-9700
checkpoint-13100  checkpoint-17300  checkpoint-6500   checkpoint-9800
checkpoint-13200  checkpoint-18100  checkpoint-6600   checkpoint-9900
checkpoint-13230  checkpoint-18200  checkpoint-6615   .gitattributes
checkpoint-14300  checkpoint-18300  checkpoint-700    README.md
checkpoint-14400  checkpoint-18400  checkpoint-7700
checkpoint-14500  checkpoint-19600  checkpoint-7800
checkpoint-14553  checkpoint-19700  checkpoint-7900


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# 16100 	0.263000 	0.552778

In [ ]:
# Start train process
trainer_stats = trainer.train(resume_from_checkpoint=True)


In [17]:
# Start train process
trainer_stats = trainer.train(resume_from_checkpoint=True)


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 14,121 | Num Epochs = 50 | Total steps = 22,050
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 10,092,544/600,000,000 (1.68% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
20800,0.149700,0.415671
20900,0.153300,0.415204
21000,0.150200,0.414333
21100,0.156100,0.412141
21200,0.153400,0.412027
21300,0.145700,0.410530
21400,0.146400,0.409665
21500,0.146200,0.407233
21600,0.146900,0.405407
21700,0.137800,0.406128


Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [18]:
!echo abc

abc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
model.save_pretrained("./Qwen-3-0.6B-Instruct-Vi-Medical-LoRA")  # Local saving
tokenizer.save_pretrained("./Qwen-3-0.6B-Instruct-Vi-Medical-LoRA")

In [21]:
prompt = ("Khi nghi ngờ bị loét dạ dày tá tràng nên đến khoa nào "
          "tại bệnh viện để thăm khám?")
          
# answer = ("Ngủ ngáy gây gián đoạn hô hấp dẫn đến mất ngủ, mệt mỏi, ảnh hưởng đến "
#           "học tập và làm việc.")

In [28]:
messages = [
    {"role" : "user", "content" : prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 2048, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.9, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Nếu nghi ngờ bị loét dạ dày tá tràng, bạn nên đến phòng khám chuyên khoa Nội khoa để được thăm khám và chẩn đoán chính xác.<|im_end|>


In [23]:
messages = [
    {"role" : "user", "content" : prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<think>

</think>

Nếu nghi ngờ bị loét dạ dày tá tràng, bạn nên đến phòng khám chuyên khoa Nội thần học để được thăm khám và chẩn đoán chính xác.<|im_end|>
